In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from datetime import date, timedelta
import pandas as pd
from numpy import datetime64

### 콜금리불러오기

In [ ]:
df_overnight=pd.read_csv("/content/drive/Shareddrives/4조/final_data/콜금리_200812_202112.csv의 사본")
df_overnight.drop(columns=[' 콜금리(1일, 전체거래)'], inplace=True)
df_overnight.columns=(['date', 'overnight'])

In [ ]:
df_overnight.dropna(inplace=True)
# df_overnight['date']= df_overnight['date'].astype('str')
# df_overnight['date'] = pd.to_datetime(df_overnight['date'])
# df_overnight.astype({'date':'datetime64'})


In [ ]:
df_overnight

,date,overnight
0,2008-12-01,3.97
1,2008-12-02,3.97
2,2008-12-03,3.97
3,2008-12-04,3.91
4,2008-12-05,3.85
...,...,...
3245,2021-12-27,1.03
3246,2021-12-28,1.03
3247,2021-12-29,1.04
3248,2021-12-30,1.12


In [ ]:
df_overnight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3250 entries, 0 to 3249
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       3250 non-null   object 
 1   overnight  3250 non-null   float64
dtypes: float64(1), object(1)
memory usage: 76.2+ KB


In [ ]:
df_date = pd.DataFrame(columns = ['date'])
start = date(2008, 12, 1)
end = date(2021, 12, 31)
for n in range(int((end - start).days) + 1):
      df_date.loc[n, 'date'] = start + timedelta(n)
df_date['date']= df_date['date'].astype('str')
df_date

,date
0,2008-12-01
1,2008-12-02
2,2008-12-03
3,2008-12-04
4,2008-12-05
...,...
4774,2021-12-27
4775,2021-12-28
4776,2021-12-29
4777,2021-12-30


In [ ]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4779 entries, 0 to 4778
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    4779 non-null   object
dtypes: object(1)
memory usage: 203.7+ KB


In [ ]:
df_date[df_date['date']=='2008-12-01']
# df_overnight[df_overnight['date']=='2008-12-01']

,date
0,2008-12-01


In [ ]:
#공휴일 콜금리 데이터 처리
def fill_call(df):
    date_data = pd.DataFrame(columns = ['date'])
    start = date(2008, 12, 1)
    end = date(2021, 12, 31)
    for n in range(int((end - start).days) + 1):
        date_data.loc[n, 'date'] = start + timedelta(n)
    date_data['date']= date_data['date'].astype('str')
    call = date_data.merge(df, on = 'date', how = 'outer')
    call['overnight'] = call['overnight'].fillna(method='ffill')
    call = call.set_index('date')

    return call

In [ ]:
df_overnight=fill_call(df_overnight)
df_overnight


,overnight
date,
2008-12-01,3.97
2008-12-02,3.97
2008-12-03,3.97
2008-12-04,3.91
2008-12-05,3.85
...,...
2021-12-27,1.03
2021-12-28,1.03
2021-12-29,1.04


### 기준금리가져오기

In [ ]:
import pandas as pd
df_baserate=pd.read_csv("/content/drive/Shareddrives/4조/final_data/BOK 기준금리_201901_202112.csv의 사본", header=None)
df_baserate.columns=(['date', 'baserate'])
df_baserate.dropna(inplace=True)
df_baserate

,date,baserate
0,2009-01-01,3.0
1,2009-01-02,3.0
2,2009-01-03,3.0
3,2009-01-04,3.0
4,2009-01-05,3.0
...,...,...
4743,2021-12-27,1.0
4744,2021-12-28,1.0
4745,2021-12-29,1.0
4746,2021-12-30,1.0


In [ ]:
df_baserate.head(10) # 기준금리는 이미 모든 일자가 있음

### 합치기

In [ ]:
df_rate = df_overnight.merge(df_baserate, on = 'date', how = 'outer')

In [ ]:
df_rate

,date,overnight,baserate
0,2008-12-01,3.97,NaN
1,2008-12-02,3.97,NaN
2,2008-12-03,3.97,NaN
3,2008-12-04,3.91,NaN
4,2008-12-05,3.85,NaN
...,...,...,...
4774,2021-12-27,1.03,1.0
4775,2021-12-28,1.03,1.0
4776,2021-12-29,1.04,1.0
4777,2021-12-30,1.12,1.0


In [ ]:
df_rate.to_csv('/content/drive/Shareddrives/4조/final_data/base_overnight.csv')

In [ ]:
df_rate=pd.read_csv('/content/drive/Shareddrives/4조/final_data/base_overnight.csv')
# df_rate['date'] = df_rate['date'].astype('datetime64')
# df_rate.set_index('date', inplace=True)
df_rate

,date,overnight,baserate
0,2008-12-01,3.97,NaN
1,2008-12-02,3.97,NaN
2,2008-12-03,3.97,NaN
3,2008-12-04,3.91,NaN
4,2008-12-05,3.85,NaN
...,...,...,...
4774,2021-12-27,1.03,1.0
4775,2021-12-28,1.03,1.0
4776,2021-12-29,1.04,1.0
4777,2021-12-30,1.12,1.0


In [ ]:
df_rate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4779 entries, 0 to 4778
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       4779 non-null   object 
 1   overnight  4779 non-null   float64
 2   baserate   4747 non-null   float64
dtypes: float64(2), object(1)
memory usage: 149.3+ KB


In [ ]:
# # 한달 전 데이터와 비교하여 상승 / 0 / 하락 라벨링
# 블로그 코드
# def up_down_call(df):
#     # df['date']= df['date'].astype('str')
#     # df['date'] = pd.to_datetime(df['date'])
#     # df.astype({'date':'datetime64'})
#     # df['date'] = df['date'].astype('datetime64')
#     # df.set_index('date', inplace=True)
#     df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
#     for i in range(len(df.index)):
#         time = df.index[i]
#         time2 = df.index[i] - pd.DateOffset(months = 1)

#         if time.year >= 2008:

#             while time2.date() not in df.index:
#                 time2 = time2 - pd.DateOffset(days = 1)

#             if df.loc[time].overnight > df.loc[time2.date()].overnight:
#                 df_ud['up_down'][i] = '상승'
#             elif df.loc[time].overnight == df.loc[time2.date()].overnight:
#                 df_ud['up_down'][i] = 0
#             else:
#                 df_ud['up_down'][i] = '하락'

#     call = df.join(df_ud, on = 'date', how = 'outer')
#     call = call.loc[date(2008, 12, 1):]
#     return call

In [ ]:
def up_down_rate(df):
    if 'date' in df.columns:
        df['date'] = df['date'].astype('datetime64')
        df.set_index('date', inplace=True)

    df_ud = pd.DataFrame(index=df.index, columns=['call_up_down', 'base_up_down' ])

    min_date = df.index.min()

    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months=1)

        if time.year >= 2008:
            while time2 not in df.index:
                if time2 < min_date:
                    break
                time2 = time2 - pd.DateOffset(days=1)

            if time2 < min_date:
                continue

            if df.loc[time].overnight > df.loc[time2].overnight:
                df_ud['call_up_down'][i] = '상승'
                if df.loc[time].baserate > df.loc[time2].baserate:
                    df_ud['base_up_down'][i] = '상승'
                elif df.loc[time].baserate == df.loc[time2].baserate:
                    df_ud['base_up_down'][i] = 0
                else:
                    df_ud['base_up_down'][i] = '하락'
            elif df.loc[time].overnight == df.loc[time2].overnight:
                df_ud['call_up_down'][i] = 0
                if df.loc[time].baserate > df.loc[time2].baserate:
                    df_ud['base_up_down'][i] = '상승'
                elif df.loc[time].baserate == df.loc[time2].baserate:
                    df_ud['base_up_down'][i] = 0
                else:
                    df_ud['base_up_down'][i] = '하락'
            else:
                df_ud['call_up_down'][i] = '하락'
                if df.loc[time].baserate > df.loc[time2].baserate:
                    df_ud['base_up_down'][i] = '상승'
                elif df.loc[time].baserate == df.loc[time2].baserate:
                    df_ud['base_up_down'][i] = 0
                else:
                    df_ud['base_up_down'][i] = '하락'

    call = df.join(df_ud, on='date', how='outer')
    call = call.loc[date(2005, 1, 1):]
    return call


In [ ]:
# call = up_down_call(df_rate)
abc = up_down_rate(df_rate)
# call = up_down_rate_final(df_rate)

In [ ]:
abc

,overnight,baserate,call_up_down,base_up_down
date,,,,
2008-12-01,3.97,NaN,NaN,NaN
2008-12-02,3.97,NaN,NaN,NaN
2008-12-03,3.97,NaN,NaN,NaN
2008-12-04,3.91,NaN,NaN,NaN
2008-12-05,3.85,NaN,NaN,NaN
...,...,...,...,...
2021-12-27,1.03,1.0,상승,0
2021-12-28,1.03,1.0,상승,0
2021-12-29,1.04,1.0,상승,0


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(abc, *['overnight'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(abc, *['baserate'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(abc, *['overnight'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(abc, *['baserate'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def heatmap(df, x_colname, y_colname, figscale=1, mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=(8 * figscale, 8 * figscale))
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(abc, *['call_up_down', 'base_up_down'], **{})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(abc, *['overnight', 'call_up_down'], **{'inner': 'box'})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(abc, *['overnight', 'base_up_down'], **{'inner': 'box'})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(abc, *['baserate', 'call_up_down'], **{'inner': 'box'})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(abc, *['baserate', 'base_up_down'], **{'inner': 'box'})
chart

In [ ]:
abc.to_csv('/content/drive/Shareddrives/4조/final_data/labeling_overnight_base.csv')

### only overnight rate

In [148]:
def up_down_overnight_final(df):
    if 'date' in df.columns:
        df['date'] = df['date'].astype('datetime64')
        df.set_index('date', inplace=True)

    df_ud = pd.DataFrame(index=df.index, columns=['up_down'])

    min_date = df.index.min()

    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months=1)

        if time.year >= 2005:
            while time2 not in df.index:
                if time2 < min_date:
                    break
                time2 = time2 - pd.DateOffset(days=1)

            if time2 < min_date:
                continue

            if df.loc[time].overnight > df.loc[time2].overnight:
                df_ud['up_down'][i] = '상승'
            elif df.loc[time].overnight == df.loc[time2].overnight:
                df_ud['up_down'][i] = 0
            else:
                df_ud['up_down'][i] = '하락'

    overnight = df.join(df_ud, on='date', how='outer')
    overnight = overnight.loc[date(2005, 1, 1):]
    return overnight

In [149]:
df_rate=pd.read_csv('/content/drive/Shareddrives/4조/final_data/base_overnight.csv')

In [150]:
df_rate.drop(columns=['baserate'], inplace=True)

In [151]:
df_rate = up_down_overnight_final(df_rate)
df_rate

,overnight,up_down
date,,
2008-12-01,3.97,NaN
2008-12-02,3.97,NaN
2008-12-03,3.97,NaN
2008-12-04,3.91,NaN
2008-12-05,3.85,NaN
...,...,...
2021-12-27,1.03,상승
2021-12-28,1.03,상승
2021-12-29,1.04,상승


In [153]:
df_rate= df_rate[31:]

,overnight,up_down
date,,
2009-01-01,3.02,하락
2009-01-02,2.87,하락
2009-01-03,2.87,하락
2009-01-04,2.87,하락
2009-01-05,2.75,하락
...,...,...
2021-12-27,1.03,상승
2021-12-28,1.03,상승
2021-12-29,1.04,상승


In [154]:
df_rate.to_csv('/content/drive/Shareddrives/4조/final_data/labeling_overnight_only.csv')